In [1]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_csv('Books.csv')

C:\Users\saadz\AppData\Local\Temp\ipykernel_2160\679519066.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [3]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [4]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [5]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [6]:
books.rename(columns={'Book-Title' : 'title', 'Book-Author' : 'author', 'Year-Of-Publication' : 'year', 'Publisher' : 'publisher'}, inplace=True)

In [7]:
books.head(2)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [8]:
users = pd.read_csv('Users.csv')

In [9]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [10]:
users.rename(columns={'User-ID' : 'user_id', 'Location' : 'location', 'Age' : 'age'}, inplace=True)

In [11]:
users.head(2)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [12]:
ratings = pd.read_csv('Ratings.csv')

In [13]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [14]:
ratings.rename(columns={'User-ID' : 'user_id', 'Book-Rating' : 'rating'}, inplace=True)

In [15]:
ratings.head(2)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5


In [16]:
books.shape

(271360, 5)

In [17]:
users.shape

(278858, 3)

In [18]:
ratings.shape

(1149780, 3)

# "Collaborative Recommendation":
278K users => Picking the users who gave ratings on atleast 200 books. 
270K books => Selecting  the books with most ratings or min_rating = 50

In [19]:
x = ratings['user_id'].value_counts()>200 # ==> give boolean answere

In [20]:
# selecting only with TRUE boolean value
# x[x].shape
x[x].index # index ==> returns id of users we need

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [21]:
y = x[x].index

In [22]:
ratings = ratings[ratings['user_id'].isin(y)]

In [23]:
ratings.shape

(526356, 3)

In [24]:
ratings_with_books = ratings.merge(books, on='ISBN')

In [25]:
ratings_with_books.shape

(487671, 7)

In [26]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [27]:
number_rating.rename(columns= {'rating' : 'number of ratings'}, inplace=True)

In [28]:
number_rating

,title,number of ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160264,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
160265,Ã?Â?lpiraten.,1
160266,Ã?Â?rger mit Produkt X. Roman.,1
160267,Ã?Â?stlich der Berge.,1


In [29]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [31]:
final_rating.shape

(487671, 8)

In [32]:
final_rating = final_rating[final_rating['number of ratings'] >= 50]

In [33]:
final_rating.shape

(61853, 8)

In [34]:
final_rating.drop_duplicates(['user_id', 'title'], inplace=True)

C:\Users\saadz\AppData\Local\Temp\ipykernel_2160\3662504876.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_rating.drop_duplicates(['user_id', 'title'], inplace=True)


In [35]:
final_rating.shape

(59850, 8)

In [36]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')

In [38]:
book_pivot.fillna(0, inplace=True)

In [39]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [41]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [42]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')

In [43]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [50]:
distances, suggestions = model.kneighbors(book_pivot.iloc[536, :].values.reshape(1,-1), n_neighbors=6)

In [51]:
suggestions

array([[536, 184, 187, 291, 204, 597]], dtype=int64)

In [52]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['The Cradle Will Fall', 'Exclusive', 'Eyes of a Child',
       'Jacob Have I Loved', 'Fine Things', 'The Long Road Home'],
      dtype='object', name='title')


In [53]:
book_pivot.index[536]

'The Cradle Will Fall'

In [56]:
# extracting ID of book for recommender
np.where(book_pivot.index=='Animal Farm')[0][0] # [0][0] ==> 2d matrix

54

In [61]:
def recommend_book(book):
    book_id = np.where(book_pivot.index=='Animal Farm')[0][0]
    distances, suggestions = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1), n_neighbors=6)
    
    for i in range(len(suggestions)):
        if i == 0:
            print('The Suggestions for ', book, 'are : ')
        if not i:
            print(book_pivot.index[suggestions[i]])    

In [62]:
recommend_book('Animal Farm')

The Suggestions for  Animal Farm are : 
Index(['Animal Farm', 'Exclusive', 'Jacob Have I Loved', 'Second Nature',
       'Pleading Guilty', 'No Safe Place'],
      dtype='object', name='title')
